In [1]:
import numpy as np

from collections import namedtuple

In [2]:
'''
struct to storage order price and volume to trade

Usage:
a = Order(price=1, volume=1)
'''

Order = namedtuple('order', 'id price volume')

In [3]:
class dynamic_order:
    '''
    extension of Order, used in making deals
    '''
    def __init__(self, order):
        assert isinstance(order, Order)
        self.id = order.id
        self.price = order.price
        self.volume = order.volume
    @property
    def show(self):
        print('id : %d' % self.id)
        print('price : %d' % self.price)
        print('volume : %d' % self.volume)

In [4]:
class Trader:
    '''
    A primary class to define an investor
    '''
    def __init__(self, id):
        self.id = id
        self.asset = 0
        self.stock = 0
        self.order = Order(id=self.id,
                           price=0,
                           volume=0)
        
    def gen_order(self):
        self.order = Order(id=self.id,
                           price=np.random.randint(-10, 10), 
                           volume=np.random.randint(1, 5))
        
    def update(self, price, vol):
        self.asset -= price * vol
        self.stock += np.sign(price) * vol
        
    @property
    def show(self):
        print('id : %d' % self.id)
        print('asset : %d' % self.asset)
        print('stock : %d' % self.stock)

In [5]:
investors = [Trader(i) for i in range(20)]

In [6]:
orders = []
for investor in investors:
    investor.gen_order()
    orders.append(investor.order)

In [7]:
orders

[order(id=0, price=4, volume=1),
 order(id=1, price=9, volume=2),
 order(id=2, price=2, volume=2),
 order(id=3, price=7, volume=4),
 order(id=4, price=5, volume=4),
 order(id=5, price=-7, volume=2),
 order(id=6, price=5, volume=2),
 order(id=7, price=-1, volume=3),
 order(id=8, price=7, volume=4),
 order(id=9, price=2, volume=4),
 order(id=10, price=-7, volume=1),
 order(id=11, price=-10, volume=1),
 order(id=12, price=7, volume=2),
 order(id=13, price=-10, volume=2),
 order(id=14, price=-3, volume=2),
 order(id=15, price=6, volume=3),
 order(id=16, price=8, volume=4),
 order(id=17, price=-2, volume=4),
 order(id=18, price=2, volume=2),
 order(id=19, price=-8, volume=1)]

In [8]:
def make_deals(orders):
    buy_list = [dynamic_order(order) for order in orders if order.price>0]
    sell_list = [dynamic_order(order) for order in orders if order.price<0]

    buy_list = sorted(buy_list, key=lambda x:(-x.price, x.id))
    sell_list = sorted(sell_list, key=lambda x:(abs(x.price), x.id))
    deal_result = np.zeros(20, int)
    price = 0
    if len(buy_list)==0 or len(sell_list)==0:
        return price, deal_result
    while sell_list[0].price+buy_list[0].price>=0:
        if sell_list[0].volume>=buy_list[0].volume:
            sell_list[0].volume -= buy_list[0].volume
            deal_result[buy_list[0].id] += buy_list[0].volume
            deal_result[sell_list[0].id] += buy_list[0].volume
            if sell_list[0].volume==buy_list[0].volume:
                price = (buy_list[0].price + sell_list[0].price) / 2
            else:
                price = sell_list[0].price
            _ = buy_list.pop(0)
        else:
            buy_list[0].volume -= sell_list[0].volume
            deal_result[buy_list[0].id] += sell_list[0].volume
            deal_result[sell_list[0].id] += sell_list[0].volume
            price = buy_list[0].price
            _ = sell_list.pop(0)           
        price = abs(price)
        if len(buy_list)==0 or len(sell_list)==0:
            break
    return price, deal_result

In [9]:
price, deal_result = make_deals(orders)
deal_result

array([0, 2, 0, 4, 0, 2, 0, 3, 2, 0, 1, 0, 0, 0, 2, 0, 4, 4, 0, 0])

In [10]:
for i,investor in enumerate(investors):
    investor.update(price=price, vol=deal_result[i])